In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
from autotransformer import *
spt = AutoTransformer()

In [3]:
# Inputs for the function
frequency = 50  # Hz
temperature_rise_goal = 100  # celcius
output_power = 40000  # watts
input_voltage = 230 # volts
output_voltage = 230  # volts
efficiency = 99  # %
regulation = 5
b_ac = 1.25  # flux density
current_density = 300  # amp/cm2
Core_Loss_Factor = 1.5 
bobbin_thickness = 1.5  # mm
insulation_thickness = 0.2  # mm
Resistivity_conductor = 1.68 * 10**-6 # ohm cm

In [4]:
# from auto transformer
k_f = spt.k_f
k_u = spt.k_u
lamination_data = spt.lamination_data
swg_data = spt.swg_data
strip_data = spt.strip_data

In [5]:
print(spt.core_loss_factor(frequency, b_ac)) 

0.6030834723022851


In [6]:
strip_data

,combination,Normal Conductor Area mm²,Conductor Weight for 1000m/Kg,height,weight
0,3x3,9.00000,79.470,3.300,3.300
1,4x3,12.00000,105.960,3.300,4.300
2,5x3,15.00000,132.450,3.300,5.300
3,3x3x2,18.00000,158.940,6.600,3.300
4,3x3x3,27.00000,238.410,9.900,3.300
5,4x3x2,24.00000,211.920,6.600,4.300
6,4x3x3,36.00000,317.880,9.900,4.300
7,4x3x4,48.00000,423.840,13.200,4.300
8,5x3x2,30.00000,264.900,6.600,5.300
9,5x3x3,45.00000,397.350,9.900,5.300


In [7]:
# calculate the apparent power
apparent_power = spt.apparent_power(output_power, efficiency)
apparent_power

80404.04040404041

In [8]:
# area product
area_product = spt.area_product(apparent_power,b_ac, current_density,frequency, k_f, k_u)
area_product = area_product * 10**4
area_product

241453574.78690806

In [9]:
##############################################################
#                       Primary wire
# calculate the input current
input_current = output_power / input_voltage
# bare area in mm2
a_wp = spt.bare_area(input_current, current_density)
# for primary wire
required_strip_primary, actual_a_wp, height_priamry, width_primary = spt.find_strip_lamination(a_wp)

# d_wp = diameter_of_primary_wire
print(input_current)
print(height_priamry)
print(required_strip_primary)

173.91304347826087
13.2
   combination  Normal Conductor Area mm²  Conductor Weight for 1000m/Kg  \
10       5x3x4                       60.0                          529.8   

    height  weight  
10    13.2     5.3  


In [10]:
##############################################################
#                     Secondary Wire
# calculate secondary current
secondary_current = output_power / output_voltage
# bare area secondary in mm2
a_ws = spt.bare_area(secondary_current, current_density)
# for secondary wire
# required_swg_secondary, diameter_of_secondary_wire, actual_a_ws = spt.find_swg(a_ws)

required_strip_secondary, actual_a_ws, height_secondary, width_secondary = spt.find_strip_lamination(a_ws)

In [11]:
# %%
def strip_cost(stack, tongue, winding_width, winding_lenghth):
    
    if stack < 5 * tongue:

        A_c = spt.core_area(stack, tongue)

        # ************************ Primary Wire ******************************** 

        Number_of_primary_turns = spt.primary_turns(input_voltage, b_ac, frequency, A_c)

        Number_of_primary_turns = round(Number_of_primary_turns)

        Turns_per_layer_primary = math.floor(spt.turns_per_layer_strip(winding_lenghth, width_primary))

        Number_of_layers_primary = math.ceil(spt.number_of_layers(Number_of_primary_turns, Turns_per_layer_primary))

        Built_primary = spt.built_primary_strip(Number_of_layers_primary, height_priamry, insulation_thickness)

        MTL_primary = spt.mtl_primary(tongue, stack, bobbin_thickness, Built_primary)

        Length_primary = spt.length(MTL_primary, Number_of_primary_turns)

        Primary_resistance = spt.resistance(Resistivity_conductor, Length_primary, actual_a_wp)

        Primary_copper_loss = spt.conductor_loss(input_current, Primary_resistance)

        # ************************ Primary Wire ******************************** 

        # ************************ Secondary Wire ******************************

        Number_of_secondary_turns = spt.secondary_turns(Number_of_primary_turns, output_voltage, regulation, input_voltage)

        Number_of_secondary_turns = round(Number_of_secondary_turns)

        Turns_per_layer_secondary = math.floor(spt.turns_per_layer_strip(winding_lenghth, width_secondary))

        Number_of_layers_secondary = math.ceil(spt.number_of_layers(Number_of_secondary_turns, Turns_per_layer_secondary))

        Built_secondary = spt.built_secondary_strip(Number_of_layers_secondary, height_secondary, insulation_thickness)

        MTL_secondary = spt.mtl_secondary(tongue, stack, Built_primary, Built_secondary, bobbin_thickness)

        Length_secondary = spt.length(MTL_secondary, Number_of_secondary_turns)

        Secondary_resistance = spt.resistance(Resistivity_conductor, Length_secondary, actual_a_ws)

        Secondary_copper_loss = spt.conductor_loss(secondary_current, Secondary_resistance)

        # ************************ Secondary Wire ******************************

        Weight_of_copper_kg = (Length_primary * required_strip_primary['Conductor Weight for 1000m/Kg'].max() + Length_secondary * required_strip_secondary['Conductor Weight for 1000m/Kg'].max() ) / 10**5  #kg

        Total_Built = spt.total_built(Built_primary, Built_secondary, bobbin_thickness)

        if winding_width * 0.9 > Total_Built:

            Total_Cu_loss = spt.total_copper_loss(Primary_copper_loss, Secondary_copper_loss)

            # Core_loss_factor = spt.core_loss_factor(frequency, b_ac) 
            Core_loss_factor = Core_Loss_Factor

            volume_of_core = spt.volume_of_core(stack, tongue, winding_width, winding_lenghth)

            Density_of_core = 7.65 # g/cm^3

            weight_of_core = spt.weight_of_core(Density_of_core, volume_of_core)

            weight_of_core_kg = weight_of_core / 1000  # kg

            core_loss = spt.core_loss(Core_loss_factor, weight_of_core_kg) 

            total_loss = spt.total_loss(Total_Cu_loss, core_loss)

            conductor_surface_area = spt.conductor_surface_area(stack, Total_Built, tongue, winding_lenghth)  # cm2

            core_surface_area = spt.core_surface_area(stack, tongue, winding_lenghth, winding_width)  # cm2

            total_surface_area = spt.total_surface_area(stack, tongue, winding_lenghth, winding_width, Total_Built)  # cm2

            psi_copper = spt.psi(Total_Cu_loss, conductor_surface_area)

            temperature_rise_copper = spt.temperature_rise(psi_copper)

            psi_core = spt.psi(core_loss, core_surface_area)

            temperature_rise_core = spt.temperature_rise(psi_core)

            cost = spt.cost(weight_of_core_kg, Weight_of_copper_kg, rate_copper=950, rate_fe=160)

            results_data = {
                    'Stack mm': stack,
                    'Tongue mm': tongue,
                    'Winding width mm': winding_width,
                    'Winding lenght mm': winding_lenghth,
                    'Length primary': Length_primary,
                    'Length secondary': Length_secondary,
                    'Cu surface area': conductor_surface_area,
                    'Core surface area': core_surface_area,
                    'Weight of Fe': weight_of_core_kg,
                    'Weight of Cu kg': Weight_of_copper_kg,
                    'Cu Temperature rise': temperature_rise_copper,
                    'Fe Temperature rise': temperature_rise_core,
                    'Cost': cost
                }

            return results_data 

In [12]:
strip_data_result= []

"""
write down the strip algorithm under this cell

Area = s * t * ww * wl

s, t, ww, wl muliples of 5

s * t * ww * wl  <= Area_product

95 < stack < 400
120 < t < 250
20 < ww < 250
120 < wl < 600

s = [50, 55, 60 ... 445]
t = [50, 55, .. ]

example stack_cost(50, 200, 600, 50)
"""
for stack in range(95, 400, 5):
    for tongue in range(120, 250, 5):
        if stack < 3 * tongue and stack > 0.8 * tongue:
            for ww in range(20, 250, 5):
                for wl in range(120, 600, 5):
                    product = stack * tongue * ww * wl 
                    if area_product * 1.5 > product and area_product * 0.5 < product:
                        results = strip_cost(stack, tongue, ww, wl)
                        if results:
                            strip_data_result.append(results)

final_result = pd.DataFrame(strip_data_result)

In [13]:
final_result

,Stack mm,Tongue mm,Winding width mm,Winding lenght mm,Length primary,Length secondary,Cu surface area,Core surface area,Weight of Fe,Weight of Cu kg,Cu Temperature rise,Fe Temperature rise,Cost
0,100,120,35,385,3447.24,3988.8,2602.6,4222.0,96.169680,39.396140,139.378406,27.667847,52813.481724
1,100,120,35,390,3447.24,3988.8,2636.4,4256.0,97.060140,39.396140,137.900770,27.695190,52955.955324
2,100,120,35,395,3447.24,3988.8,2670.2,4290.0,97.950600,39.396140,136.457325,27.722094,53098.428924
3,100,120,35,400,3447.24,3988.8,2704.0,4324.0,98.841060,39.396140,135.046864,27.748570,53240.902524
4,100,120,35,405,3447.24,3988.8,2737.8,4358.0,99.731520,39.396140,133.668233,27.774627,53383.376124
...,...,...,...,...,...,...,...,...,...,...,...,...,...
138982,395,200,35,130,1249.60,1304.0,1853.8,9240.0,427.940235,13.528973,76.292545,49.721166,81322.961760
138983,395,205,35,120,1259.60,1314.0,1723.2,9311.5,432.629991,13.634933,81.561912,49.852391,82173.984720
138984,395,205,35,125,1259.60,1314.0,1795.0,9392.0,438.638741,13.634933,78.857578,50.066368,83135.384700
138985,395,210,35,120,1269.60,1324.0,1735.2,9544.0,449.337247,13.740893,81.615928,50.400098,84947.807640


In [14]:
final_result.sort_values('Cost')

,Stack mm,Tongue mm,Winding width mm,Winding lenght mm,Length primary,Length secondary,Cu surface area,Core surface area,Weight of Fe,Weight of Cu kg,Cu Temperature rise,Fe Temperature rise,Cost
96994,225,125,35,160,2202.84,2442.0,1497.6,4322.5,133.569000,24.608362,149.153915,35.594235,44748.984204
102124,235,120,35,160,2231.84,2472.0,1513.6,4285.0,131.832603,24.920944,149.399926,35.465931,44768.113584
96995,225,125,35,165,2202.84,2442.0,1544.4,4370.0,135.656016,24.608362,145.410586,35.729002,45082.906704
97036,225,125,40,160,2202.84,2442.0,1497.6,4392.5,135.656016,24.608362,149.153915,35.577763,45082.906704
106936,245,120,35,155,2210.88,2447.6,1497.3,4354.5,135.260874,24.680627,149.540359,35.747781,45088.335528
...,...,...,...,...,...,...,...,...,...,...,...,...,...
134214,340,245,35,120,1229.60,1284.0,1687.2,10210.0,494.502120,13.317053,81.395204,51.593089,91771.539360
137125,365,235,35,120,1259.60,1314.0,1723.2,10199.5,496.464842,13.634933,81.561912,51.806188,92387.560920
136167,355,240,35,120,1249.60,1304.0,1711.2,10253.0,499.459014,13.528973,81.507130,51.839629,92765.966400
134900,345,245,35,120,1239.60,1294.0,1699.2,10302.5,501.774210,13.423013,81.451567,51.831428,93035.735760
